In [2]:
import add_path
import torch
import torch.nn as nn
import torch.nn.functional as F
import pickle

In [3]:
from transformers import pipeline
from transformers.pipelines import PIPELINE_REGISTRY
from models.BERT.speaker_tokens import SpeakerBertTokenizer
from models.BERT.tucorecgcn_bert_pytorch_pipeline import ConversationalSequenceClassificationPipeline
from models.BERT.tucoregcn_bert_pytorch import TUCOREGCN_BertForSequenceClassification, TUCOREGCN_BertConfig
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
PIPELINE_REGISTRY.register_pipeline(
    "conversational-sequence-classification",
    pipeline_class=ConversationalSequenceClassificationPipeline
)
speaker_tokenizer = SpeakerBertTokenizer.from_pretrained('bert-base-uncased')
model = TUCOREGCN_BertForSequenceClassification(TUCOREGCN_BertConfig.from_json_file("../models/BERT/tucoregcn_bert_mlc.json"))
model.cuda()
model.load_state_dict(torch.load("../TUCOREGCN_BERT_DialogRE/tucoregcn_pytorch_model.pt"))
model.cuda()
classifier = pipeline("conversational-sequence-classification", model=model, tokenizer=speaker_tokenizer, device="cuda:0")

d:\projects\affect\TUCORE-GCN\.venv\Lib\site-packages\dgl\dgl.dll


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'SpeakerBertTokenizer'.


In [37]:
from models.BERT.tucoregcn_bert_pytorch_processor import SpeakerRelation, Conversation, Message

c = Conversation(
	messages=[
		Message("Speaker 1", "Howdy! I'm Flowey, Flowey the Flower!"),
		Message("Speaker 2", "Hello Flowey."),
	],
	speaker_relations=[
		SpeakerRelation("Speaker 1", "Speaker 2")
	]
)
labels, scores, logits = classifier(c).values()
print(labels, scores)

['LABEL_14'] [0.38450223207473755]
